## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Alfred,ZA,-33.5906,26.8910,70.66,82,69,3.78,broken clouds
1,1,Puerto Ayora,EC,-0.7393,-90.3518,78.55,93,96,3.49,overcast clouds
2,2,Broome,US,42.2506,-75.8330,38.01,70,100,5.01,overcast clouds
3,3,Cherskiy,RU,68.7500,161.3000,18.43,76,90,5.68,overcast clouds
4,4,Prince Albert,CA,53.2001,-105.7677,-2.25,50,40,17.27,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? ")) 

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10) 


,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Puerto Ayora,EC,-0.7393,-90.3518,78.55,93,96,3.49,overcast clouds
6,6,Kapaa,US,22.0752,-159.3190,81.12,71,20,10.36,few clouds
7,7,Rikitea,PF,-23.1203,-134.9692,78.75,70,59,5.55,broken clouds
11,11,Alofi,NU,-19.0595,-169.9187,78.69,89,75,8.05,broken clouds
20,20,Arraial Do Cabo,BR,-22.9661,-42.0278,77.67,85,2,23.87,clear sky
22,22,Celestun,MX,20.8667,-90.4000,77.74,85,37,8.12,scattered clouds
23,23,Ayame,CI,5.6083,-3.1563,76.64,92,83,2.62,broken clouds
26,26,Mahebourg,MU,-20.4081,57.7000,79.09,89,40,3.44,scattered clouds
39,39,Hithadhoo,MV,-0.6000,73.0833,82.00,60,7,4.79,clear sky
47,47,Sinnamary,GF,5.3833,-52.9500,76.95,86,100,8.10,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.count() 

171


Unnamed: 0             171
City                   171
Country                170
Lat                    171
Lng                    171
Max Temp               171
Humidity               171
Cloudiness             171
Wind Speed             171
Current Description    171
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head() 

170


,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Puerto Ayora,EC,-0.7393,-90.3518,78.55,93,96,3.49,overcast clouds
6,6,Kapaa,US,22.0752,-159.3190,81.12,71,20,10.36,few clouds
7,7,Rikitea,PF,-23.1203,-134.9692,78.75,70,59,5.55,broken clouds
11,11,Alofi,NU,-19.0595,-169.9187,78.69,89,75,8.05,broken clouds
20,20,Arraial Do Cabo,BR,-22.9661,-42.0278,77.67,85,2,23.87,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,78.55,overcast clouds,-0.7393,-90.3518,
6,Kapaa,US,81.12,few clouds,22.0752,-159.3190,
7,Rikitea,PF,78.75,broken clouds,-23.1203,-134.9692,
11,Alofi,NU,78.69,broken clouds,-19.0595,-169.9187,
20,Arraial Do Cabo,BR,77.67,clear sky,-22.9661,-42.0278,
22,Celestun,MX,77.74,scattered clouds,20.8667,-90.4000,
23,Ayame,CI,76.64,broken clouds,5.6083,-3.1563,
26,Mahebourg,MU,79.09,scattered clouds,-20.4081,57.7000,
39,Hithadhoo,MV,82.00,clear sky,-0.6000,73.0833,
47,Sinnamary,GF,76.95,overcast clouds,5.3833,-52.9500,


In [27]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:   
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
       
        print("Hotel not found... skipping.") 
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [28]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10) 

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,78.55,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
6,Kapaa,US,81.12,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
7,Rikitea,PF,78.75,broken clouds,-23.1203,-134.9692,People ThankYou
11,Alofi,NU,78.69,broken clouds,-19.0595,-169.9187,Taloa Heights
20,Arraial Do Cabo,BR,77.67,clear sky,-22.9661,-42.0278,Pousada Porto Praia
22,Celestun,MX,77.74,scattered clouds,20.8667,-90.4000,Posada Lilia
23,Ayame,CI,76.64,broken clouds,5.6083,-3.1563,Hotel Bossan
26,Mahebourg,MU,79.09,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
39,Hithadhoo,MV,82.00,clear sky,-0.6000,73.0833,Scoop Guest House
47,Sinnamary,GF,76.95,overcast clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary


In [29]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [30]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))